# Imports


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import numpy as np
print("Torch Version:",torch.__version__)
from torch import nn
import torch.nn.functional as F
from torchvision import datasets,transforms
import torch.optim as optim

Torch Version: 1.4.0


In [2]:
from classification_report import Report, Config, HyperParameters

# Hyperparameters

In [3]:
h_p = HyperParameters(training_config = Config(batch_size=64 , lr=0.01 , no_epochs=10),
                      inference_config = Config(batch_size=64),
                      model_config = Config(no_of_channels=10 ,classes = ["zero","one","two","three","four","five","six","seven","eight","nine"])
                     )


In [4]:
print(f"Training batch size {h_p.training_config.batch_size}")
print(f"Number of classes {h_p.model_config.classes.__len__()}") 

Training batch size 64
Number of classes 10


# Datasets and Dataloaders

In [5]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))]
                            )

trainset=datasets.MNIST('~/.pytorch/MNIST_data/',
                        train=True,
                        transform=transform,
                        download=True)

validset=datasets.MNIST('~/.pytorch/MNIST_data/',
                        train=False,
                        transform=transform,
                        download=True)

train_loader=torch.utils.data.DataLoader(trainset,
                                         batch_size=h_p.training_config.batch_size,
                                         shuffle=True,
                                         num_workers=0)

valid_loader=torch.utils.data.DataLoader(validset,
                                         batch_size=h_p.inference_config.batch_size,
                                         shuffle=True,
                                         num_workers=0)

# Model Defination

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 
                               h_p.model_config.no_of_channels,
                               3)  #[in_channel,out_channel,filter_size,stride=1]
        
        self.fc2 = nn.Linear(8*8*h_p.model_config.no_of_channels,
                             h_p.model_config.classes.__len__())

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,3)
        x = x.view(-1, 8*8*10)
        x = self.fc2(x)
        return x # return raw logits.
      
# create a complete CNN
model = Net()
print(model)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (fc2): Linear(in_features=640, out_features=10, bias=True)
)


In [7]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Total_params",pytorch_total_params)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable_params",pytorch_total_params)

Total_params 6510
Trainable_params 6510


# Report Intialization

In [8]:
data , label = next(iter(train_loader))
print(f"The first input batch shape {data.shape}")

The first input batch shape torch.Size([64, 1, 28, 28])


In [9]:
report = Report(classes=h_p.model_config.classes) # intialize the class
report.plot_model(model, data) # plot model

# Criterion and Optimizer

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=h_p.training_config.lr)

# Training Method

In [11]:
def train_an_epoch():
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
        # converting raw _logits to softmax output
        output = F.softmax(output,dim=-1)
        
        # write training batch information into report
        report.write_a_batch(loss=loss,
                             batch_size=data.size(0),
                             actual=target,
                             prediction=output,
                             train=True)
        
        #plot histogram of model weight, bias and gradients 2 times in an epoch
        report.plot_model_data_grad(at_which_iter = len(train_loader)/2)
        
    return train_loss

# Validation Method

In [12]:
def valid_an_epoch():
        
    valid_loss = 0.0
    model.eval()
    for data, target in valid_loader:
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        
        
        # converting raw _logits to softmax output
        output = F.softmax(output,dim=-1)
        
        # write validation batch information into report
        report.write_a_batch(loss=loss,
                             batch_size=data.size(0),
                             actual=target,
                             prediction=output,
                             train=False)
    return valid_loss

# Training and Plotting

In [13]:
for epoch in range(1, h_p.training_config.no_epochs+1):

    train_loss = train_an_epoch()
    valid_loss = valid_an_epoch()
    
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # print training/validation statistics 
    print(f'Epoch: {epoch} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f}')
    
    report.plot_an_epoch() # plot after every epoch


Epoch: 1 	Training Loss: 0.476556 	Validation Loss: 0.281798
Epoch: 2 	Training Loss: 0.266338 	Validation Loss: 0.215414
Epoch: 3 	Training Loss: 0.207621 	Validation Loss: 0.174787
Epoch: 4 	Training Loss: 0.168733 	Validation Loss: 0.145775
Epoch: 5 	Training Loss: 0.143169 	Validation Loss: 0.125264
Epoch: 6 	Training Loss: 0.124155 	Validation Loss: 0.107546
Epoch: 7 	Training Loss: 0.111645 	Validation Loss: 0.096930
Epoch: 8 	Training Loss: 0.102040 	Validation Loss: 0.090832
Epoch: 9 	Training Loss: 0.094731 	Validation Loss: 0.084824
Epoch: 10 	Training Loss: 0.089033 	Validation Loss: 0.079835


# Plot Hyperparams

In [14]:
report.plot_hparams(h_p) # record hyper params once training is completed

#    Last Epoch Statistics

In [15]:
report.loss_count # final loss for 10th epoch 

{'train': 0.08903284033934275, 'valid': 0.07983549206852913}

In [16]:
report.act_pred_dict # actual and prediction for 10th epoch

{'train': {'actual': array([2, 9, 6, ..., 6, 5, 6]),
  'pred': array([[2.5042798e-05, 6.5744323e-05, 9.3698359e-01, ..., 1.9802920e-04,
          7.8922687e-03, 5.7031157e-05],
         [1.9366228e-06, 7.1479769e-09, 2.3671055e-05, ..., 1.6210197e-03,
          4.4759642e-04, 9.9449807e-01],
         [1.1239527e-05, 1.0241215e-03, 3.8109050e-05, ..., 5.5505899e-07,
          9.4072195e-04, 1.0471222e-06],
         ...,
         [1.8088377e-04, 2.5010223e-07, 1.9928522e-03, ..., 4.1664250e-09,
          7.3243369e-05, 6.3713242e-06],
         [6.4380409e-08, 1.2906545e-08, 3.9848260e-08, ..., 1.0480987e-06,
          1.3894854e-04, 2.0053969e-05],
         [2.9811055e-05, 4.6260371e-05, 1.6806077e-04, ..., 2.0354689e-07,
          2.0149567e-04, 3.4741282e-07]], dtype=float32)},
 'valid': {'actual': array([2, 5, 0, ..., 9, 7, 2]),
  'pred': array([[3.0130930e-05, 3.4288573e-07, 9.8805797e-01, ..., 2.2048336e-07,
          1.1788650e-02, 5.2077816e-05],
         [6.5836525e-07, 4.3492469

In [17]:
report.counter # number of epoch

10

In [18]:
report.iter_count # train_loader and valid_loader size

Counter({'train': 9380, 'valid': 1570})

In [19]:
report.data_count # data point count

Counter({'train': 60000, 'valid': 10000})

In [20]:
report.mcc # Mathews correlation coefficient for 10th epoch

Counter({'train': 0.9719173510233347, 'valid': 0.973886044489139})

In [21]:
report.close() # close the writer object.